In [25]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [13]:
#function  to create a server connection

def server_conn(host_name,user_name,user_password):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name,
                                            user = user_name,
                                            passwd = user_password)
        print("Server is connected")
    except Error as err:
        print("Error - ", err)
    return connection

In [4]:
connection = server_conn("127.0.0.1","root","725@Mysql")
print(connection)

Server is connected


In [8]:
#function to create database
def create_db(connection,db_name):
    cursor = connection.cursor()
    try:
        query = f"create database if not exists {db_name}"
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print("Error-", err)

In [10]:
#to create a database
connection = server_conn("127.0.0.1","root","725@Mysql")
database_name = "employee_delhi"
create_db(connection,database_name)

Server is connected
Database created successfully


In [12]:
#function to connect with database

def db_conn(host_name,user_name,user_password,db_name):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name,
                                            user = user_name,
                                            passwd = user_password,
                                            database = db_name)
        print("Database is connected")
    except Error as err:
        print("Error - ", err)
    return connection

In [14]:
connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
print(connection)

Database is connected


In [15]:
#function to execute query
def execute_query(connection,query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as err:
        print("Error - ", err)

In [16]:
#create an employee order table

create_ord_table = """
create table emp_orders(
e_id int primary key,
e_name varchar(30),
date DATE,
e_dept varchar(25),
e_gender varchar(25),
quantity int,
amount decimal(7,2)
)
"""

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
execute_query(connection,create_ord_table)

Database is connected
Query executed successfully


In [19]:
#insert data into table
order_info = """
insert into emp_orders (e_id,e_name,date,e_dept,e_gender,quantity,amount)
values
(1,"John", "2023-08-05" , "Production" , "Male" , 2 , 2500.50),
(2,"Amrita", "2023-12-04" , "Research" , "Female" , 3 , 4500.50),
(5,"Sameera", "2024-01-05" , "Production" , "Female" , 1 , 1000.50),
(7,"Jessica", "2023-09-15" , "Development" , "Female" , 5 , 5000.50),
(6,"Ajay", "2024-05-05" , "Development" , "Male" , 2 , 2200.50),
(4,"Anil", "2024-04-03" , "Research" , "Male" , 1 , 900.50)
"""

In [20]:
connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
execute_query(connection,order_info)

Database is connected
Query executed successfully


In [22]:
#read function

def read_table(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
    except Error as err:
        print("Error - ", err)
    return result

In [23]:
query = "select * from emp_orders"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , query)
for res in results:
    print(res)

Database is connected
(1, 'John', datetime.date(2023, 8, 5), 'Production', 'Male', 2, Decimal('2500.50'))
(2, 'Amrita', datetime.date(2023, 12, 4), 'Research', 'Female', 3, Decimal('4500.50'))
(4, 'Anil', datetime.date(2024, 4, 3), 'Research', 'Male', 1, Decimal('900.50'))
(5, 'Sameera', datetime.date(2024, 1, 5), 'Production', 'Female', 1, Decimal('1000.50'))
(6, 'Ajay', datetime.date(2024, 5, 5), 'Development', 'Male', 2, Decimal('2200.50'))
(7, 'Jessica', datetime.date(2023, 9, 15), 'Development', 'Female', 5, Decimal('5000.50'))


In [27]:

emp_orders_info = []
for res in results:
    emp_orders_info.append(res)

column_names = ['emp_id','Name','Date','Department','Gender','Quantity','Amount']

df = pd.DataFrame(emp_orders_info , columns = column_names)
df

,emp_id,Name,Date,Department,Gender,Quantity,Amount
0,1,John,2023-08-05,Production,Male,2,2500.50
1,2,Amrita,2023-12-04,Research,Female,3,4500.50
2,4,Anil,2024-04-03,Research,Male,1,900.50
3,5,Sameera,2024-01-05,Production,Female,1,1000.50
4,6,Ajay,2024-05-05,Development,Male,2,2200.50
5,7,Jessica,2023-09-15,Development,Female,5,5000.50


In [28]:
df.groupby('Department')['Amount'].agg(['mean','min','max','sum','count'])

,mean,min,max,sum,count
Department,,,,,
Development,3600.5,2200.50,5000.50,7201.00,2
Production,1750.5,1000.50,2500.50,3501.00,2
Research,2700.5,900.50,4500.50,5401.00,2


In [29]:
min_amount = "select min(amount) from emp_orders"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , min_amount)
for res in results:
    print(res)

Database is connected
(Decimal('900.50'),)


In [30]:
average_amount = "select avg(amount) from emp_orders"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , average_amount)
for res in results:
    print(res)

Database is connected
(Decimal('2683.833333'),)


In [32]:
average_amount = "select e_dept,avg(amount) from emp_orders group by e_dept"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , average_amount)
for res in results:
    print(res)

Database is connected
('Production', Decimal('1750.500000'))
('Research', Decimal('2700.500000'))
('Development', Decimal('3600.500000'))


In [40]:
max_amount = "select e_dept,max(amount) from emp_orders group by e_dept"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , max_amount)
for res in results:
    print(res)

Database is connected
('Production', Decimal('2500.50'))
('Research', Decimal('4500.50'))
('Development', Decimal('5000.50'))


In [41]:
max_amount = "select e_gender,max(amount) from emp_orders group by e_gender"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , max_amount)
for res in results:
    print(res)

Database is connected
('Male', Decimal('2500.50'))
('Female', Decimal('5000.50'))


In [42]:
count = "select e_gender,count(*) from emp_orders group by e_gender"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , count)
for res in results:
    print(res)

Database is connected
('Male', 3)
('Female', 3)


In [43]:
count = "select e_dept,count(*) from emp_orders group by e_dept"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , count)
for res in results:
    print(res)

Database is connected
('Production', 2)
('Research', 2)
('Development', 2)


In [44]:
count = "select e_dept,count(*) from emp_orders where e_dept = 'Research'"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , count)
for res in results:
    print(res)

Database is connected
('Research', 2)


In [46]:
#'mean','min','max','sum','count'

analysis = "select e_dept, avg(amount),min(amount),max(amount),sum(amount),count(*) from emp_orders group by e_dept"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , analysis)
for res in results:
    print(res)

Database is connected
('Production', Decimal('1750.500000'), Decimal('1000.50'), Decimal('2500.50'), Decimal('3501.00'), 2)
('Research', Decimal('2700.500000'), Decimal('900.50'), Decimal('4500.50'), Decimal('5401.00'), 2)
('Development', Decimal('3600.500000'), Decimal('2200.50'), Decimal('5000.50'), Decimal('7201.00'), 2)


In [47]:
l_case = "select lower(e_name) from emp_orders"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , l_case)
for res in results:
    print(res)

Database is connected
('john',)
('amrita',)
('anil',)
('sameera',)
('ajay',)
('jessica',)


In [48]:
three_chars = "select substring(e_name,1,3) from emp_orders"

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , three_chars)
for res in results:
    print(res)

Database is connected
('Joh',)
('Amr',)
('Ani',)
('Sam',)
('Aja',)
('Jes',)


In [49]:
case_query = """
select *,
case
when amount < 1000 then 'C'
when amount < 3000 then 'B'
else 'A'
end as BAND
from emp_orders;
"""

connection = db_conn("127.0.0.1","root","725@Mysql","employee_delhi")
results = read_table(connection , case_query)
for res in results:
    print(res)

Database is connected
(1, 'John', datetime.date(2023, 8, 5), 'Production', 'Male', 2, Decimal('2500.50'), 'B')
(2, 'Amrita', datetime.date(2023, 12, 4), 'Research', 'Female', 3, Decimal('4500.50'), 'A')
(4, 'Anil', datetime.date(2024, 4, 3), 'Research', 'Male', 1, Decimal('900.50'), 'C')
(5, 'Sameera', datetime.date(2024, 1, 5), 'Production', 'Female', 1, Decimal('1000.50'), 'B')
(6, 'Ajay', datetime.date(2024, 5, 5), 'Development', 'Male', 2, Decimal('2200.50'), 'B')
(7, 'Jessica', datetime.date(2023, 9, 15), 'Development', 'Female', 5, Decimal('5000.50'), 'A')


**having vs wher